In [13]:
import pandas as pd


In [14]:
# read in data
wedstrijden = pd.read_excel('../../Data/Silver/wedstrijden_cleaned.xlsx')
corners = pd.read_excel('../../Data/Silver/cornerballen_cleaned.xlsx')

controle_wed = pd.read_excel('../../Data/Bronze/wedstrijden_controle.xlsx')
controle_corners = pd.read_excel('../../Data/Bronze/corners_controle.xlsx')

In [15]:
merged_df = pd.merge(wedstrijden, corners, on='wedstrijd_id', how='left')

merged_controle = pd.merge(controle_wed, controle_corners, on='wedstrijd_id', how='left')

In [16]:
merged_controle.columns

Index(['wedstrijd_id', 'datum', 'niveau', 'uitslag', 'corners_uitslag',
       'klassement_uitslag', 'schepcorner_id', 'ploegnaam',
       'ervaring_schepper', 'ervaring_kopper', 'ervaring_duo', 'is_thuisploeg',
       'kwart', 'is_schepper_bank', 'is_kopper_bank', 'score_voor',
       'schepper_pos_rechts', 'positie', 'kop_verplaatsen_achter',
       'kop_veel_verplaatsen', 'kop_hor_verplaatsen', 'goal'],
      dtype='object')

### controle van enquetekwaliteit

In [17]:
# wedstrijd_ids voor matching wedstrijden eruit krijgen
matching = pd.merge(merged_df, merged_controle, on=["datum", "ploegnaam"], how="inner")
matching


,wedstrijd_id_x,datum,niveau_x,uitslag_x,corners_uitslag_x,klassement_uitslag_x,schepcorner_id_x,ploegnaam,ervaring_schepper_x,ervaring_kopper_x,...,kwart_y,is_schepper_bank_y,is_kopper_bank_y,score_voor_y,schepper_pos_rechts_y,positie_y,kop_verplaatsen_achter_y,kop_veel_verplaatsen_y,kop_hor_verplaatsen_y,goal_y
0,9,2024-09-02,1,4v5,1v1,8v15,22.0,citadel park rangers,NaN,NaN,...,2,0,0,0v2,1,2,0,0,0,1
1,9,2024-09-02,1,4v5,1v1,8v15,22.0,citadel park rangers,NaN,NaN,...,3,0,0,1v2,1,2,0,0,0,1
2,9,2024-09-02,1,4v5,1v1,8v15,23.0,citadel park rangers,NaN,NaN,...,2,0,0,0v2,1,2,0,0,0,1
3,9,2024-09-02,1,4v5,1v1,8v15,23.0,citadel park rangers,NaN,NaN,...,3,0,0,1v2,1,2,0,0,0,1
4,32,2025-04-02,4,6v4,2v1,3v4,76.0,fc spitbulls,3.0,3.0,...,2,0,0,1v0,0,2,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,171,2024-12-12,2,3v10,2v0,14v11,441.0,vvks sint lieven,NaN,NaN,...,3,0,0,3v8,1,5,0,1,1,0
81,171,2024-12-12,2,3v10,2v0,14v11,442.0,spectex,NaN,NaN,...,1,1,0,0v2,1,5,0,0,0,1
82,171,2024-12-12,2,3v10,2v0,14v11,442.0,spectex,NaN,NaN,...,2,0,0,1v4,1,5,0,1,1,1
83,171,2024-12-12,2,3v10,2v0,14v11,442.0,spectex,NaN,NaN,...,4,1,0,3v9,1,5,0,0,1,1


In [18]:


# met de bijhorende wedstrijdids uit df1
wedstrijdids = matching["wedstrijd_id_x"].unique()

In [19]:
print(len(wedstrijdids))
wedstrijdids

20


array([  9,  32,  33,  34,  77,  90,  91, 106, 107, 138, 140, 159, 160,
       161, 165, 167, 169, 170, 171, 172], dtype=int64)

In [20]:
### filter op basis van wedstrijdids
merged_filtered = merged_df[merged_df["wedstrijd_id"].isin(wedstrijdids)].reset_index()
wedstijden_filtered = wedstrijden[wedstrijden["wedstrijd_id"].isin(wedstrijdids)].reset_index()
corners_filtered = corners[corners["wedstrijd_id"].isin(wedstrijdids)].reset_index()

In [21]:
# zorg dat df in zelfde volgorde staan, geen wedstrijden met zelfde niveau en datum
merged_filtered = merged_filtered.sort_values(by=["datum", "niveau", "schepcorner_id"]).reset_index()
merged_controle = merged_controle.sort_values(by=["datum", "niveau", "schepcorner_id"]).reset_index()

In [ ]:
# bekeijk distributie van kop_veel_verplaatsen, want lage cohen's kappa score
print(merged_filtered["kop_veel_verplaatsen"].value_counts())
print(merged_controle["kop_veel_verplaatsen"].value_counts())


kop_veel_verplaatsen
0.0    41
1.0    13
Name: count, dtype: int64
kop_veel_verplaatsen
0    46
1     8
Name: count, dtype: int64


In [10]:
merged_filtered.head()

,level_0,index,wedstrijd_id,datum,niveau,uitslag,corners_uitslag,klassement_uitslag,schepcorner_id,ploegnaam,...,kwart,is_schepper_bank,is_kopper_bank,score_voor,schepper_pos_rechts,positie,kop_verplaatsen_achter,kop_veel_verplaatsen,kop_hor_verplaatsen,goal
0,0,21,9,2024-09-02,1,4v5,1v1,8v15,22.0,citadel park rangers,...,2.0,0.0,0.0,0v2,1.0,2.0,0.0,0.0,0.0,1.0
1,1,22,9,2024-09-02,1,4v5,1v1,8v15,23.0,citadel park rangers,...,3.0,0.0,0.0,1v2,1.0,2.0,0.0,0.0,0.0,1.0
2,2,23,9,2024-09-02,1,4v5,1v1,8v15,24.0,expats gent fc,...,4.0,0.0,0.0,4v4,1.0,2.0,0.0,0.0,0.0,0.0
3,18,229,91,2024-09-10,3,7v4,2v0,NaN,229.0,devaldo rojo,...,2.0,0.0,0.0,2v1,1.0,5.0,0.0,0.0,0.0,1.0
4,19,230,91,2024-09-10,3,7v4,2v0,NaN,230.0,devaldo rojo,...,4.0,0.0,0.0,7v3,1.0,5.0,0.0,0.0,0.0,1.0


In [11]:
# functie voor aantal matchende waarde te checken in de kolommen
def compare_column_values(df1, df2, column):
    # Compare values
    matches = df1[column] == df2[column]
    num_matches = matches.sum()
    total = len(matches)
    percentage = (num_matches / total) * 100

    print(f"Percentage same value: {percentage:.2f}%")
    print(f"Number of matching values: {num_matches} out of {total}")

    # Print mismatches
    if num_matches < total:
        print("\nMismatched rows:")
        mismatch_indices = matches[~matches].index
        for i in mismatch_indices:
            print(f"\nRow {i}:")
            print("df1:", df1.loc[i].to_dict())
            print("df2:", df2.loc[i].to_dict())


In [12]:
merged_controle.columns

Index(['index', 'wedstrijd_id', 'datum', 'niveau', 'uitslag',
       'corners_uitslag', 'klassement_uitslag', 'schepcorner_id', 'ploegnaam',
       'ervaring_schepper', 'ervaring_kopper', 'ervaring_duo', 'is_thuisploeg',
       'kwart', 'is_schepper_bank', 'is_kopper_bank', 'score_voor',
       'schepper_pos_rechts', 'positie', 'kop_verplaatsen_achter',
       'kop_veel_verplaatsen', 'kop_hor_verplaatsen', 'goal'],
      dtype='object')

In [13]:
compare_column_values(merged_filtered, merged_controle, "niveau")

Percentage same value: 100.00%
Number of matching values: 54 out of 54


In [14]:
compare_column_values(merged_filtered, merged_controle, "uitslag")

Percentage same value: 100.00%
Number of matching values: 54 out of 54


In [15]:
compare_column_values(merged_filtered, merged_controle, "corners_uitslag")

Percentage same value: 85.19%
Number of matching values: 46 out of 54

Mismatched rows:

Row 8:
df1: {'level_0': 20, 'index': 274, 'wedstrijd_id': 106, 'datum': Timestamp('2024-09-11 00:00:00'), 'niveau': 1, 'uitslag': '6v4', 'corners_uitslag': nan, 'klassement_uitslag': nan, 'schepcorner_id': 274.0, 'ploegnaam': 'girafkes', 'ervaring_schepper': nan, 'ervaring_kopper': nan, 'ervaring_duo': nan, 'is_thuisploeg': 0.0, 'kwart': 3.0, 'is_schepper_bank': 1.0, 'is_kopper_bank': 0.0, 'score_voor': '2v2', 'schepper_pos_rechts': 1.0, 'positie': 5.0, 'kop_verplaatsen_achter': 0.0, 'kop_veel_verplaatsen': 1.0, 'kop_hor_verplaatsen': 0.0, 'goal': 0.0}
df2: {'index': 8, 'wedstrijd_id': 4, 'datum': Timestamp('2024-09-11 00:00:00'), 'niveau': 1, 'uitslag': '6v4', 'corners_uitslag': '3v1', 'klassement_uitslag': nan, 'schepcorner_id': 9, 'ploegnaam': 'girafkes', 'ervaring_schepper': nan, 'ervaring_kopper': nan, 'ervaring_duo': nan, 'is_thuisploeg': 0, 'kwart': 3, 'is_schepper_bank': 1, 'is_kopper_bank'

In [16]:
compare_column_values(merged_filtered, merged_controle, "kwart")

Percentage same value: 98.15%
Number of matching values: 53 out of 54

Mismatched rows:

Row 12:
df1: {'level_0': 13, 'index': 195, 'wedstrijd_id': 77, 'datum': Timestamp('2024-09-23 00:00:00'), 'niveau': 4, 'uitslag': '4v10', 'corners_uitslag': '2v3', 'klassement_uitslag': nan, 'schepcorner_id': 195.0, 'ploegnaam': 'de plasticboys', 'ervaring_schepper': 5.0, 'ervaring_kopper': 5.0, 'ervaring_duo': 5.0, 'is_thuisploeg': 0.0, 'kwart': 2.0, 'is_schepper_bank': 0.0, 'is_kopper_bank': 0.0, 'score_voor': '2v6', 'schepper_pos_rechts': 1.0, 'positie': 5.0, 'kop_verplaatsen_achter': 0.0, 'kop_veel_verplaatsen': 0.0, 'kop_hor_verplaatsen': 0.0, 'goal': 1.0}
df2: {'index': 12, 'wedstrijd_id': 6, 'datum': Timestamp('2024-09-23 00:00:00'), 'niveau': 4, 'uitslag': '4v10', 'corners_uitslag': '1v3', 'klassement_uitslag': nan, 'schepcorner_id': 13, 'ploegnaam': 'de plasticboys', 'ervaring_schepper': nan, 'ervaring_kopper': nan, 'ervaring_duo': nan, 'is_thuisploeg': 0, 'kwart': 3, 'is_schepper_bank': 0

In [17]:
compare_column_values(merged_filtered, merged_controle, "is_schepper_bank")

Percentage same value: 98.15%
Number of matching values: 53 out of 54

Mismatched rows:

Row 27:
df1: {'level_0': 42, 'index': 432, 'wedstrijd_id': 169, 'datum': Timestamp('2024-12-05 00:00:00'), 'niveau': 3, 'uitslag': '6v10', 'corners_uitslag': '1v0', 'klassement_uitslag': '8v5', 'schepcorner_id': 432.0, 'ploegnaam': 'mp trois', 'ervaring_schepper': 3.0, 'ervaring_kopper': 3.0, 'ervaring_duo': 2.0, 'is_thuisploeg': 1.0, 'kwart': 3.0, 'is_schepper_bank': 0.0, 'is_kopper_bank': 0.0, 'score_voor': '1v4', 'schepper_pos_rechts': 1.0, 'positie': 1.0, 'kop_verplaatsen_achter': 0.0, 'kop_veel_verplaatsen': 0.0, 'kop_hor_verplaatsen': 0.0, 'goal': 1.0}
df2: {'index': 27, 'wedstrijd_id': 12, 'datum': Timestamp('2024-12-05 00:00:00'), 'niveau': 3, 'uitslag': '6v10', 'corners_uitslag': '1v0', 'klassement_uitslag': nan, 'schepcorner_id': 28, 'ploegnaam': 'mp trois', 'ervaring_schepper': nan, 'ervaring_kopper': nan, 'ervaring_duo': nan, 'is_thuisploeg': 1, 'kwart': 3, 'is_schepper_bank': 1, 'is_ko

In [18]:
compare_column_values(merged_filtered, merged_controle, "is_kopper_bank")

Percentage same value: 100.00%
Number of matching values: 54 out of 54


In [19]:
compare_column_values(merged_filtered, merged_controle, "score_voor")

Percentage same value: 98.15%
Number of matching values: 53 out of 54

Mismatched rows:

Row 28:
df1: {'level_0': 43, 'index': 433, 'wedstrijd_id': 169, 'datum': Timestamp('2024-12-05 00:00:00'), 'niveau': 3, 'uitslag': '6v10', 'corners_uitslag': '1v0', 'klassement_uitslag': '8v5', 'schepcorner_id': 433.0, 'ploegnaam': 'de daltons', 'ervaring_schepper': nan, 'ervaring_kopper': nan, 'ervaring_duo': nan, 'is_thuisploeg': 0.0, 'kwart': 4.0, 'is_schepper_bank': 0.0, 'is_kopper_bank': 0.0, 'score_voor': '4v7', 'schepper_pos_rechts': 1.0, 'positie': 2.0, 'kop_verplaatsen_achter': 1.0, 'kop_veel_verplaatsen': 0.0, 'kop_hor_verplaatsen': 0.0, 'goal': 1.0}
df2: {'index': 28, 'wedstrijd_id': 12, 'datum': Timestamp('2024-12-05 00:00:00'), 'niveau': 3, 'uitslag': '6v10', 'corners_uitslag': '1v0', 'klassement_uitslag': nan, 'schepcorner_id': 29, 'ploegnaam': 'de daltons', 'ervaring_schepper': nan, 'ervaring_kopper': nan, 'ervaring_duo': nan, 'is_thuisploeg': 0, 'kwart': 4, 'is_schepper_bank': 0, 'i

In [20]:
compare_column_values(merged_filtered, merged_controle, "schepper_pos_rechts")

Percentage same value: 100.00%
Number of matching values: 54 out of 54


In [21]:
compare_column_values(merged_filtered, merged_controle, "positie")

Percentage same value: 85.19%
Number of matching values: 46 out of 54

Mismatched rows:

Row 3:
df1: {'level_0': 18, 'index': 229, 'wedstrijd_id': 91, 'datum': Timestamp('2024-09-10 00:00:00'), 'niveau': 3, 'uitslag': '7v4', 'corners_uitslag': '2v0', 'klassement_uitslag': nan, 'schepcorner_id': 229.0, 'ploegnaam': 'devaldo rojo', 'ervaring_schepper': 5.0, 'ervaring_kopper': 5.0, 'ervaring_duo': 5.0, 'is_thuisploeg': 0.0, 'kwart': 2.0, 'is_schepper_bank': 0.0, 'is_kopper_bank': 0.0, 'score_voor': '2v1', 'schepper_pos_rechts': 1.0, 'positie': 5.0, 'kop_verplaatsen_achter': 0.0, 'kop_veel_verplaatsen': 0.0, 'kop_hor_verplaatsen': 0.0, 'goal': 1.0}
df2: {'index': 3, 'wedstrijd_id': 2, 'datum': Timestamp('2024-09-10 00:00:00'), 'niveau': 3, 'uitslag': '7v4', 'corners_uitslag': '2v0', 'klassement_uitslag': nan, 'schepcorner_id': 4, 'ploegnaam': 'devaldo rojo', 'ervaring_schepper': nan, 'ervaring_kopper': nan, 'ervaring_duo': nan, 'is_thuisploeg': 0, 'kwart': 2, 'is_schepper_bank': 0, 'is_kop

In [22]:
compare_column_values(merged_filtered, merged_controle, "kop_verplaatsen_achter")

Percentage same value: 98.15%
Number of matching values: 53 out of 54

Mismatched rows:

Row 17:
df1: {'level_0': 32, 'index': 413, 'wedstrijd_id': 160, 'datum': Timestamp('2024-10-29 00:00:00'), 'niveau': 4, 'uitslag': '2v5', 'corners_uitslag': '0v1', 'klassement_uitslag': '6v7', 'schepcorner_id': 413.0, 'ploegnaam': 'red star ghent', 'ervaring_schepper': 5.0, 'ervaring_kopper': 5.0, 'ervaring_duo': 5.0, 'is_thuisploeg': 1.0, 'kwart': 2.0, 'is_schepper_bank': 0.0, 'is_kopper_bank': 0.0, 'score_voor': '0v1', 'schepper_pos_rechts': 1.0, 'positie': 5.0, 'kop_verplaatsen_achter': 1.0, 'kop_veel_verplaatsen': 0.0, 'kop_hor_verplaatsen': 0.0, 'goal': 0.0}
df2: {'index': 17, 'wedstrijd_id': 8, 'datum': Timestamp('2024-10-29 00:00:00'), 'niveau': 4, 'uitslag': '2v5', 'corners_uitslag': '0v1', 'klassement_uitslag': nan, 'schepcorner_id': 18, 'ploegnaam': 'red star ghent', 'ervaring_schepper': nan, 'ervaring_kopper': nan, 'ervaring_duo': nan, 'is_thuisploeg': 1, 'kwart': 2, 'is_schepper_bank': 

In [23]:
compare_column_values(merged_filtered, merged_controle, "kop_veel_verplaatsen")

Percentage same value: 87.04%
Number of matching values: 47 out of 54

Mismatched rows:

Row 8:
df1: {'level_0': 20, 'index': 274, 'wedstrijd_id': 106, 'datum': Timestamp('2024-09-11 00:00:00'), 'niveau': 1, 'uitslag': '6v4', 'corners_uitslag': nan, 'klassement_uitslag': nan, 'schepcorner_id': 274.0, 'ploegnaam': 'girafkes', 'ervaring_schepper': nan, 'ervaring_kopper': nan, 'ervaring_duo': nan, 'is_thuisploeg': 0.0, 'kwart': 3.0, 'is_schepper_bank': 1.0, 'is_kopper_bank': 0.0, 'score_voor': '2v2', 'schepper_pos_rechts': 1.0, 'positie': 5.0, 'kop_verplaatsen_achter': 0.0, 'kop_veel_verplaatsen': 1.0, 'kop_hor_verplaatsen': 0.0, 'goal': 0.0}
df2: {'index': 8, 'wedstrijd_id': 4, 'datum': Timestamp('2024-09-11 00:00:00'), 'niveau': 1, 'uitslag': '6v4', 'corners_uitslag': '3v1', 'klassement_uitslag': nan, 'schepcorner_id': 9, 'ploegnaam': 'girafkes', 'ervaring_schepper': nan, 'ervaring_kopper': nan, 'ervaring_duo': nan, 'is_thuisploeg': 0, 'kwart': 3, 'is_schepper_bank': 1, 'is_kopper_bank'

In [24]:
compare_column_values(merged_filtered, merged_controle, "kop_hor_verplaatsen")

Percentage same value: 90.74%
Number of matching values: 49 out of 54

Mismatched rows:

Row 17:
df1: {'level_0': 32, 'index': 413, 'wedstrijd_id': 160, 'datum': Timestamp('2024-10-29 00:00:00'), 'niveau': 4, 'uitslag': '2v5', 'corners_uitslag': '0v1', 'klassement_uitslag': '6v7', 'schepcorner_id': 413.0, 'ploegnaam': 'red star ghent', 'ervaring_schepper': 5.0, 'ervaring_kopper': 5.0, 'ervaring_duo': 5.0, 'is_thuisploeg': 1.0, 'kwart': 2.0, 'is_schepper_bank': 0.0, 'is_kopper_bank': 0.0, 'score_voor': '0v1', 'schepper_pos_rechts': 1.0, 'positie': 5.0, 'kop_verplaatsen_achter': 1.0, 'kop_veel_verplaatsen': 0.0, 'kop_hor_verplaatsen': 0.0, 'goal': 0.0}
df2: {'index': 17, 'wedstrijd_id': 8, 'datum': Timestamp('2024-10-29 00:00:00'), 'niveau': 4, 'uitslag': '2v5', 'corners_uitslag': '0v1', 'klassement_uitslag': nan, 'schepcorner_id': 18, 'ploegnaam': 'red star ghent', 'ervaring_schepper': nan, 'ervaring_kopper': nan, 'ervaring_duo': nan, 'is_thuisploeg': 1, 'kwart': 2, 'is_schepper_bank': 

In [25]:
compare_column_values(merged_filtered, merged_controle, "goal")

Percentage same value: 100.00%
Number of matching values: 54 out of 54


### Cohen's kappa
most of the times used to describe raters agreement

In [26]:
from sklearn.metrics import cohen_kappa_score

In [27]:
print(merged_filtered["corners_uitslag"].isna().sum())
print(merged_controle["corners_uitslag"].isna().sum())

3
0


In [28]:
merged_filtered["corners_uitslag"] = merged_filtered["corners_uitslag"].fillna("NA")

In [33]:
print(merged_filtered["niveau"].isna().sum())
merged_controle["niveau"].isna().sum()

0


0

In [34]:
merged_filtered['niveau']

0      1
1      1
2      1
3      3
4      3
5      4
6      4
7      4
8      1
9      3
10     3
11     4
12     4
13     4
14     4
15     4
16     4
17     4
18     4
19     3
20     3
21     3
22     2
23     2
24     4
25     4
26     4
27     3
28     3
29     3
30     4
31     4
32     4
33     2
34     2
35     2
36     2
37     2
38     4
39    n1
40    n1
41    n1
42    n1
43    n4
44    n4
45     4
46     4
47     4
48     4
49     4
50     4
51     4
52     4
53     4
Name: niveau, dtype: object

In [35]:
merged_controle["niveau"]

0      1
1      1
2      1
3      3
4      3
5      4
6      4
7      4
8      1
9      3
10     3
11     4
12     4
13     4
14     4
15     4
16     4
17     4
18     4
19     3
20     3
21     3
22     2
23     2
24     4
25     4
26     4
27     3
28     3
29     3
30     4
31     4
32     4
33     2
34     2
35     2
36     2
37     2
38     4
39    n1
40    n1
41    n1
42    n1
43    n4
44    n4
45     4
46     4
47     4
48     4
49     4
50     4
51     4
52     4
53     4
Name: niveau, dtype: object

In [37]:
niveau1 = merged_filtered["niveau"].astype(str).str.strip()
niveau2 = merged_controle["niveau"].astype(str).str.strip()

# Reset index om uitlijning zeker te stellen
niveau1 = niveau1.reset_index(drop=True)
niveau2 = niveau2.reset_index(drop=True)

# Safety check
assert len(niveau1) == len(niveau2)


In [40]:
print("Cohen's kappa score voor niveau", cohen_kappa_score(niveau1, niveau2))
print("Cohen's kappa score voor uitslag", cohen_kappa_score(merged_filtered["uitslag"], merged_controle["uitslag"]))
print("Cohen's kappa score voor corners uitslag", cohen_kappa_score(merged_filtered["corners_uitslag"], merged_controle["corners_uitslag"]))
print("Cohen's kappa score voor kwart", cohen_kappa_score(merged_filtered["kwart"], merged_controle["kwart"]))
print("Cohen's kappa score voor is_schepper_bank", cohen_kappa_score(merged_filtered["is_schepper_bank"], merged_controle["is_schepper_bank"]))
print("Cohen's kappa score voor is_kopper_bank", cohen_kappa_score(merged_filtered["is_kopper_bank"], merged_controle["is_kopper_bank"]))
print("Cohen's kappa score voor score_voor", cohen_kappa_score(merged_filtered["score_voor"], merged_controle["score_voor"]))
print("Cohen's kappa score voor schepper_pos_rechts", cohen_kappa_score(merged_filtered["schepper_pos_rechts"], merged_controle["schepper_pos_rechts"]))
print("Cohen's kappa score voor positie", cohen_kappa_score(merged_filtered["positie"], merged_controle["positie"]))
print("Cohen's kappa score voor kop_verplaatsen_achter", cohen_kappa_score(merged_filtered["kop_verplaatsen_achter"], merged_controle["kop_verplaatsen_achter"]))
print("Cohen's kappa score voor kop_veel_verplaatsen", cohen_kappa_score(merged_filtered["kop_veel_verplaatsen"], merged_controle["kop_veel_verplaatsen"]))
print("Cohen's kappa score voor kop_hor_verplaatsen", cohen_kappa_score(merged_filtered["kop_hor_verplaatsen"], merged_controle["kop_hor_verplaatsen"]))
print("Cohen's kappa score voor goal", cohen_kappa_score(merged_filtered["goal"], merged_controle["goal"]))

Cohen's kappa score voor niveau 1.0
Cohen's kappa score voor uitslag 1.0
Cohen's kappa score voor corners uitslag 0.8341650671785029
Cohen's kappa score voor kwart 0.9731075697211156
Cohen's kappa score voor is_schepper_bank 0.9226361031518625
Cohen's kappa score voor is_kopper_bank 1.0
Cohen's kappa score voor score_voor 0.9805124503789245
Cohen's kappa score voor schepper_pos_rechts 1.0
Cohen's kappa score voor positie 0.7244897959183674
Cohen's kappa score voor kop_verplaatsen_achter 0.8789237668161435
Cohen's kappa score voor kop_veel_verplaatsen 0.591792656587473
Cohen's kappa score voor kop_hor_verplaatsen 0.7761194029850746
Cohen's kappa score voor goal 1.0
